# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119557e+02     1.618855e+00
 * time: 0.09425902366638184
     1     1.042719e+01     9.363134e-01
 * time: 1.04960298538208
     2    -1.125156e+01     1.092506e+00
 * time: 1.200653076171875
     3    -3.396558e+01     8.111573e-01
 * time: 1.3770740032196045
     4    -4.727855e+01     5.519689e-01
 * time: 1.5433080196380615
     5    -5.676807e+01     2.053543e-01
 * time: 1.6989390850067139
     6    -5.962820e+01     1.488611e-01
 * time: 1.813002109527588
     7    -6.077391e+01     7.079119e-02
 * time: 1.9350650310516357
     8    -6.126628e+01     4.012618e-02
 * time: 2.0640389919281006
     9    -6.154228e+01     4.581290e-02
 * time: 2.181812047958374
    10    -6.173877e+01     3.277473e-02
 * time: 2.314577102661133
    11    -6.189736e+01     2.550111e-02
 * time: 2.4303719997406006
    12    -6.199953e+01     2.101474e-02
 * time: 2.5640101432800293
    13    -6.205212e+01     1.987852e-02
 * time: 2.693958044052124

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557692
    AtomicNonlocal      14.8522659
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485375 
    Xc                  -19.3336821

    total               -62.261666460714
